In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go

#

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
data = pd.read_csv('../input/g-research-crypto-forecasting/train.csv') 
asset_details = pd.read_csv('../input/g-research-crypto-forecasting/asset_details.csv') 

In [ ]:
data.info()

In [ ]:
asset_details.info()

Just parallelised different volatitlites from
[here](https://github.com/jasonstrimpel/volatility-trading)
and made them all-in-one function

In [ ]:
def get_estimator(price_data, estimator,trading_periods,window=7*24*60):
    window=7*24*60
    if estimator == 'Parkinson':
        rs = (1.0 / (4.0 * math.log(2.0))) * ((price_data['High'] / price_data['Low']).apply(np.log)) ** 2.0
    elif estimator == 'GarmanKlass':
        log_hl = (price_data['High'] / price_data['Low']).apply(np.log)
        log_co = (price_data['Close'] / price_data['Open']).apply(np.log)
        rs = 0.5 * log_hl ** 2 - (2 * math.log(2) - 1) * log_co ** 2
    elif estimator == 'YangZhang':
        log_ho = (price_data['High'] / price_data['Open']).apply(np.log)
        log_lo = (price_data['Low'] / price_data['Open']).apply(np.log)
        log_co = (price_data['Close'] / price_data['Open']).apply(np.log)
        log_oc = (price_data['Open'] / price_data['Close'].shift(1)).apply(np.log)
        log_oc_sq = log_oc ** 2
        log_cc = (price_data['Close'] / price_data['Close'].shift(1)).apply(np.log)
        log_cc_sq = log_cc ** 2
        rs = log_ho * (log_ho - log_co) + log_lo * (log_lo - log_co)

    def f(v):
        return (trading_periods * v.mean()) ** 0.5
    
    result = pd.DataFrame()

    result['vol'] = rs.rolling(
        window=window,
        center=False
    ).parallel_apply(func=f)

    result['timestamp'] = price_data['timestamp']

    return result.dropna()

**Plot 3 types of volatilities (monthly, 7-day rolling window) for each asset out of 14**

In [ ]:
#from tqdm import tqdm


import matplotlib.pyplot as plt
from IPython.display import HTML
from matplotlib.animation import ArtistAnimation, FuncAnimation

%matplotlib inline

from pandarallel import pandarallel
pandarallel.initialize(progress_bar=False)

volatilities = {
        'YangZhang':"red",
        'GarmanKlass':"green",
        'Parkinson':"blue"
}

periods = 30*24*60

exs = data['Asset_ID'].value_counts()
print(exs)

In [ ]:
def print_vol_graph(asset, volatilities, periods):
    
    #plotly
    #fig = make_subplots(rows=3, cols=1)
    
    #matplotlib
    fig, axes = plt.subplots(nrows=3, ncols=1, sharex=True, figsize=(16,5), dpi= 80)
    fig.subplots_adjust(hspace=0.5)
    
    asset_name = asset_details.loc[asset_details['Asset_ID']==asset]['Asset_Name'].values[0]
    row_n = 0
    for vf in volatilities.keys():
        vol = get_estimator(data.loc[data['Asset_ID']==int(asset)],vf,periods)
        axes[row_n].plot(
            vol['vol'],
            color=volatilities[vf],
            linewidth=1
        )
        axes[row_n].set_ylabel(" {} ".format(vf))
        
        #fig.add_trace(go.Scatter(x=vol['timestamp'], y=vol['vol'],
        #              name = "{} volatility".format(vf),
        #              line=dict(color=volatilities[vf], width=1)),
        #              row=row_n,
        #              col=1
        #             )
        row_n+=1
    #fig.update_layout(height=700, width=900, title_text="{} volatilities".format(asset_name))
    #fig.show()
    fig.suptitle("{} volatilities".format(asset_name))
    

**for the sake of memory efficiency lets not use plotly this time**

In [ ]:
print_vol_graph(0, volatilities, periods)

In [ ]:
print_vol_graph(1, volatilities, periods)

In [ ]:
print_vol_graph(2, volatilities, periods)

In [ ]:
print_vol_graph(3, volatilities, periods)

In [ ]:
print_vol_graph(4, volatilities, periods)

In [ ]:
print_vol_graph(5, volatilities, periods)

In [ ]:
print_vol_graph(6, volatilities, periods)

In [ ]:
print_vol_graph(7, volatilities, periods)

In [ ]:
print_vol_graph(8, volatilities, periods)

In [ ]:
print_vol_graph(9, volatilities, periods)

In [ ]:
print_vol_graph(10, volatilities, periods)

In [ ]:
print_vol_graph(11, volatilities, periods)

In [ ]:
print_vol_graph(12, volatilities, periods)

In [ ]:
print_vol_graph(13, volatilities, periods)